# Risk Metrics - VaR and CVaR

Value at Risk (VaR) and Conditional Value at Risk (CVaR) analysis for the WolfpackTrend strategy.

**Data Source:**
- `{TEAM_ID}/daily_snapshots.csv` - Daily NAV for return calculation

**Analysis:**
- Historical VaR and CVaR at 95% and 99% confidence levels
- Rolling windows: 20, 60, and 252 days
- Drawdown analysis

**Prerequisites:** Run the WolfpackTrend backtest first to generate ObjectStore data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook
from config import TEAM_ID

qb = QuantBook()
print("QuantBook initialized")

## Load Data

In [ ]:
try:
    snapshots_str = qb.ObjectStore.Read(f"{TEAM_ID}/daily_snapshots.csv")
    df = pd.read_csv(StringIO(snapshots_str))
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    print(f"Loaded {len(df)} daily snapshots")
    print(f"Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    
    if 'nav' not in df.columns:
        raise ValueError("NAV column not found in daily_snapshots.csv")
        
except Exception as e:
    print(f"ERROR: {e}")
    print("Make sure you have run the WolfpackTrend backtest first.")
    df = pd.DataFrame()

## Compute Daily Returns

In [ ]:
if not df.empty:
    # Compute daily returns from NAV
    df['daily_return'] = df['nav'].pct_change()
    
    # Drop first row (NaN return)
    returns = df['daily_return'].dropna()
    
    print(f"Daily Returns Statistics:")
    print("=" * 60)
    print(f"Count: {len(returns)}")
    print(f"Mean: {returns.mean() * 100:.4f}%")
    print(f"Std: {returns.std() * 100:.4f}%")
    print(f"Min: {returns.min() * 100:.4f}%")
    print(f"Max: {returns.max() * 100:.4f}%")
    print(f"Skewness: {returns.skew():.4f}")
    print(f"Kurtosis: {returns.kurtosis():.4f}")

## Helper Functions

In [ ]:
def historical_var(returns, confidence_level=0.95):
    """
    Calculate historical Value at Risk.
    
    Args:
        returns: Series of returns
        confidence_level: Confidence level (e.g., 0.95 for 95%)
    
    Returns:
        VaR as a positive number (loss)
    """
    return -np.percentile(returns, (1 - confidence_level) * 100)


def historical_cvar(returns, confidence_level=0.95):
    """
    Calculate historical Conditional Value at Risk (Expected Shortfall).
    
    Args:
        returns: Series of returns
        confidence_level: Confidence level (e.g., 0.95 for 95%)
    
    Returns:
        CVaR as a positive number (expected loss beyond VaR)
    """
    var = historical_var(returns, confidence_level)
    return -returns[returns <= -var].mean()


def rolling_var(returns, window, confidence_level=0.95):
    """
    Calculate rolling historical VaR.
    """
    return returns.rolling(window).apply(
        lambda x: historical_var(x, confidence_level), raw=True
    )


def rolling_cvar(returns, window, confidence_level=0.95):
    """
    Calculate rolling historical CVaR.
    """
    return returns.rolling(window).apply(
        lambda x: historical_cvar(x, confidence_level), raw=True
    )


print("Helper functions defined")

## Historical VaR and CVaR

In [ ]:
if not df.empty:
    print("\nHistorical VaR and CVaR (Full Period):")
    print("=" * 60)
    
    confidence_levels = [0.95, 0.99]
    
    results = []
    for cl in confidence_levels:
        var = historical_var(returns, cl)
        cvar = historical_cvar(returns, cl)
        results.append({
            'Confidence': f"{int(cl * 100)}%",
            'VaR (daily)': f"{var * 100:.4f}%",
            'CVaR (daily)': f"{cvar * 100:.4f}%",
            'VaR (annualized)': f"{var * np.sqrt(252) * 100:.2f}%",
            'CVaR (annualized)': f"{cvar * np.sqrt(252) * 100:.2f}%"
        })
    
    results_df = pd.DataFrame(results)
    display(results_df)
    
    print("\nInterpretation:")
    print(f"  - 95% VaR: On 95% of days, daily loss should not exceed {historical_var(returns, 0.95) * 100:.4f}%")
    print(f"  - 99% VaR: On 99% of days, daily loss should not exceed {historical_var(returns, 0.99) * 100:.4f}%")
    print(f"  - 95% CVaR: When losses exceed 95% VaR, expected loss is {historical_cvar(returns, 0.95) * 100:.4f}%")

## Return Distribution

In [ ]:
if not df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    axes[0].hist(returns * 100, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    
    # Add VaR lines
    var_95 = -historical_var(returns, 0.95) * 100
    var_99 = -historical_var(returns, 0.99) * 100
    axes[0].axvline(x=var_95, color='orange', linestyle='--', linewidth=2, label=f'95% VaR: {var_95:.2f}%')
    axes[0].axvline(x=var_99, color='red', linestyle='--', linewidth=2, label=f'99% VaR: {var_99:.2f}%')
    axes[0].axvline(x=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    
    axes[0].set_title('Daily Return Distribution with VaR', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Daily Return (%)')
    axes[0].set_ylabel('Frequency')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # QQ plot
    from scipy import stats
    stats.probplot(returns, dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot (Normal Distribution)', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Rolling VaR (20/60/252 days)

In [ ]:
if not df.empty:
    windows = [20, 60, 252]
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # 95% VaR
    for window in windows:
        rolling = rolling_var(returns, window, 0.95) * 100
        axes[0].plot(df['date'].iloc[1:], rolling, linewidth=2, label=f'{window}-day', alpha=0.8)
    
    axes[0].set_title('Rolling 95% VaR', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('VaR (%)')
    axes[0].legend(loc='upper left')
    axes[0].grid(True, alpha=0.3)
    
    # 99% VaR
    for window in windows:
        rolling = rolling_var(returns, window, 0.99) * 100
        axes[1].plot(df['date'].iloc[1:], rolling, linewidth=2, label=f'{window}-day', alpha=0.8)
    
    axes[1].set_title('Rolling 99% VaR', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('VaR (%)')
    axes[1].legend(loc='upper left')
    axes[1].grid(True, alpha=0.3)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Rolling CVaR (20/60/252 days)

In [ ]:
if not df.empty:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # 95% CVaR
    for window in windows:
        rolling = rolling_cvar(returns, window, 0.95) * 100
        axes[0].plot(df['date'].iloc[1:], rolling, linewidth=2, label=f'{window}-day', alpha=0.8)
    
    axes[0].set_title('Rolling 95% CVaR (Expected Shortfall)', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('CVaR (%)')
    axes[0].legend(loc='upper left')
    axes[0].grid(True, alpha=0.3)
    
    # 99% CVaR
    for window in windows:
        rolling = rolling_cvar(returns, window, 0.99) * 100
        axes[1].plot(df['date'].iloc[1:], rolling, linewidth=2, label=f'{window}-day', alpha=0.8)
    
    axes[1].set_title('Rolling 99% CVaR (Expected Shortfall)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('CVaR (%)')
    axes[1].legend(loc='upper left')
    axes[1].grid(True, alpha=0.3)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Drawdown Analysis

In [ ]:
if not df.empty:
    # Compute cumulative returns and drawdown
    df['cumulative_return'] = (1 + df['daily_return'].fillna(0)).cumprod()
    df['running_max'] = df['cumulative_return'].cummax()
    df['drawdown'] = (df['cumulative_return'] / df['running_max']) - 1
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Cumulative returns
    axes[0].plot(df['date'], (df['cumulative_return'] - 1) * 100, linewidth=2, color='steelblue')
    axes[0].fill_between(df['date'], 0, (df['cumulative_return'] - 1) * 100, alpha=0.3)
    axes[0].set_title('Cumulative Returns', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Cumulative Return (%)')
    axes[0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0].grid(True, alpha=0.3)
    
    # Drawdown
    axes[1].fill_between(df['date'], 0, df['drawdown'] * 100, color='red', alpha=0.5)
    axes[1].plot(df['date'], df['drawdown'] * 100, linewidth=1, color='darkred')
    axes[1].set_title('Drawdown', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Drawdown (%)')
    axes[1].grid(True, alpha=0.3)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Drawdown statistics
    print("\nDrawdown Statistics:")
    print("=" * 60)
    print(f"Maximum Drawdown: {df['drawdown'].min() * 100:.2f}%")
    print(f"Average Drawdown: {df['drawdown'].mean() * 100:.2f}%")
    
    # Find max drawdown date
    max_dd_idx = df['drawdown'].idxmin()
    max_dd_date = df.loc[max_dd_idx, 'date']
    print(f"Max Drawdown Date: {max_dd_date.strftime('%Y-%m-%d')}")

## Drawdown Duration Analysis

In [ ]:
if not df.empty:
    # Identify drawdown periods
    df['in_drawdown'] = df['drawdown'] < 0
    df['drawdown_start'] = (df['in_drawdown'] & ~df['in_drawdown'].shift(1).fillna(False))
    df['drawdown_end'] = (~df['in_drawdown'] & df['in_drawdown'].shift(1).fillna(False))
    
    # Calculate drawdown durations
    drawdown_periods = []
    start_date = None
    
    for idx, row in df.iterrows():
        if row['drawdown_start']:
            start_date = row['date']
        if row['drawdown_end'] and start_date is not None:
            duration = (row['date'] - start_date).days
            max_dd = df[(df['date'] >= start_date) & (df['date'] <= row['date'])]['drawdown'].min()
            drawdown_periods.append({
                'start': start_date,
                'end': row['date'],
                'duration_days': duration,
                'max_drawdown': max_dd * 100
            })
            start_date = None
    
    if drawdown_periods:
        dd_df = pd.DataFrame(drawdown_periods)
        dd_df = dd_df.sort_values('max_drawdown')
        
        print("\nTop 10 Drawdown Periods:")
        print("=" * 80)
        display(dd_df.head(10))
        
        print(f"\nAverage drawdown duration: {dd_df['duration_days'].mean():.1f} days")
        print(f"Maximum drawdown duration: {dd_df['duration_days'].max()} days")
    else:
        print("No completed drawdown periods found")

## Rolling Volatility

In [ ]:
if not df.empty:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    for window in windows:
        rolling_vol = returns.rolling(window).std() * np.sqrt(252) * 100
        ax.plot(df['date'].iloc[1:], rolling_vol, linewidth=2, label=f'{window}-day', alpha=0.8)
    
    # Add target volatility line
    ax.axhline(y=10, color='red', linestyle='--', linewidth=2, label='Target Vol (10%)')
    
    ax.set_title('Rolling Annualized Volatility', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Volatility (%)')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Volatility statistics
    realized_vol = returns.std() * np.sqrt(252) * 100
    print(f"\nRealized Annualized Volatility: {realized_vol:.2f}%")

## Summary

In [ ]:
if not df.empty:
    print("\n" + "=" * 80)
    print("RISK METRICS SUMMARY")
    print("=" * 80)
    
    print(f"\nPeriod: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"Trading days: {len(returns)}")
    
    print(f"\nReturn Statistics:")
    print(f"  Total Return: {(df['cumulative_return'].iloc[-1] - 1) * 100:.2f}%")
    print(f"  Annualized Return: {((df['cumulative_return'].iloc[-1]) ** (252/len(returns)) - 1) * 100:.2f}%")
    print(f"  Realized Volatility: {returns.std() * np.sqrt(252) * 100:.2f}%")
    
    print(f"\nRisk Metrics (Daily):")
    print(f"  95% VaR: {historical_var(returns, 0.95) * 100:.4f}%")
    print(f"  99% VaR: {historical_var(returns, 0.99) * 100:.4f}%")
    print(f"  95% CVaR: {historical_cvar(returns, 0.95) * 100:.4f}%")
    print(f"  99% CVaR: {historical_cvar(returns, 0.99) * 100:.4f}%")
    
    print(f"\nDrawdown:")
    print(f"  Maximum Drawdown: {df['drawdown'].min() * 100:.2f}%")
    print(f"  Average Drawdown: {df['drawdown'].mean() * 100:.2f}%")
    
    print("\n" + "=" * 80)